In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# 1. 데이터 불러오기
train = pd.read_csv('../../../data/processed/train.csv')
test = pd.read_csv('../../../data/processed/test.csv')

# 불필요한 식별자 컬럼(row_id) 제거 (데이터에 존재할 경우)
if 'row_id' in train.columns:
    train = train.drop(columns=['row_id'])
if 'row_id' in test.columns:
    test = test.drop(columns=['row_id'])

# 2. 데이터 전처리 (Preprocessing)

# 수치형과 범주형 컬럼 구분
categorical_cols = ['Month', 'VisitorType', 'Weekend', 'OperatingSystems', 'Browser', 'Region', 'TrafficType']
numerical_cols = ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration', 
                  'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']

# 숫자 형태의 범주형 데이터(OS, Browser 등)를 문자열로 변환하여 카테고리로 인식하게 함
for col in ['OperatingSystems', 'Browser', 'Region', 'TrafficType']:
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)

# 원-핫 인코딩 (One-Hot Encoding)
# train과 test의 컬럼을 동일하게 맞추기 위해 합쳐서 인코딩 후 분리
train['is_train'] = 1
test['is_train'] = 0
combined = pd.concat([train, test])

# 범주형 데이터 인코딩
combined = pd.get_dummies(combined, columns=categorical_cols, drop_first=True)

# 다시 Train과 Test로 분리
train_processed = combined[combined['is_train'] == 1].drop(columns=['is_train'])
test_processed = combined[combined['is_train'] == 0].drop(columns=['is_train'])

# Feature(X)와 Target(y) 분리
X_train = train_processed.drop(columns=['Revenue'])
y_train = train_processed['Revenue'].astype(int) # 타겟을 정수형(0,1)으로 변환
X_test = test_processed.drop(columns=['Revenue'])
y_test = test_processed['Revenue'].astype(int)

# 수치형 데이터 스케일링 (StandardScaler)
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

# TensorFlow 학습을 위해 float32 타입으로 변환
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print(f"학습 데이터 형태: {X_train.shape}")
print(f"테스트 데이터 형태: {X_test.shape}")

# 3. 딥러닝 모델 구성 (Build Model)
model = Sequential([
    # 입력층 & 첫 번째 은닉층 (노드 64개)
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2), # 과적합 방지
    
    # 두 번째 은닉층 (노드 32개)
    Dense(32, activation='relu'),
    Dropout(0.2),
    
    # 세 번째 은닉층 (노드 16개)
    Dense(16, activation='relu'),
    
    # 출력층 (이진 분류이므로 노드 1개, 활성화 함수는 sigmoid)
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy', # 이진 분류 손실 함수
              metrics=['accuracy'])

# 4. 모델 학습 (Train)
history = model.fit(X_train, y_train, 
                    epochs=2000,          # 학습 반복 횟수
                    batch_size=32,      # 한 번에 학습할 데이터 크기
                    validation_split=0.2, # 학습 데이터 중 20%를 검증용으로 사용
                    verbose=1)

# 5. 성능 평가 (Evaluate)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("\n" + "="*30)
print(f"최종 테스트 정확도 (Accuracy): {accuracy:.4f}")
print(f"최종 테스트 손실값 (Loss): {loss:.4f}")
print("="*30)

# 6. 예측 결과 확인 (Prediction)
# 확률값으로 예측됨 (예: 0.85 -> Revenue가 True일 확률 85%)
predictions = model.predict(X_test)
print("\n예측값 샘플 (처음 5개):")
print(predictions[:5])

학습 데이터 형태: (9864, 68)
테스트 데이터 형태: (2466, 68)
Epoch 1/2000


/opt/miniconda3/envs/ai_basic_env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8657 - loss: 0.3575 - val_accuracy: 0.8839 - val_loss: 0.3063
Epoch 2/2000
247/247 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8799 - loss: 0.3166 - val_accuracy: 0.8819 - val_loss: 0.3067
Epoch 3/2000
196/247 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8775 - loss: 0.3268

KeyboardInterrupt: 